In [4]:
library(metacell)
library(GEOquery)
library(googlesheets4)
library(googledrive)
library(devtools)
library(dplyr)
library(tibble)
library(ggplot2)
library(tidyverse)
library(tgstat)
library(tgconfig)
library(liana)
library(SingleCellExperiment)
library(Seurat)
library(magrittr)

In [5]:
show_resources()

[1] "Default"          "Consensus"        "Baccin2019"       "CellCall"        
 [5] "CellChatDB"       "Cellinker"        "CellPhoneDB"      "CellTalkDB"      
 [9] "connectomeDB2020" "EMBRACE"          "Guide2Pharma"     "HPMR"            
[13] "ICELLNET"         "iTALK"            "Kirouac2010"      "LRdb"            
[17] "Ramilowski2015"   "OmniPath"

In [6]:
show_methods()

[1] "connectome"      "logfc"           "natmi"           "sca"            
 [5] "cellphonedb"     "cytotalk"        "call_squidpy"    "call_cellchat"  
 [9] "call_connectome" "call_sca"        "call_italk"      "call_natmi"

In [7]:
liana_path <- system.file(package = "liana")

In [8]:
op_resource <- select_resource("Consensus")[[1]]

In [9]:
ortholog_resource <- generate_homologs(op_resource = op_resource,
                                       .missing_fun = str_to_title,
                                       max_homologs = 5,
                                       target_organism = 10090) # mouse

Warning message:
“Automatic coercion from integer to character was deprecated in purrr 1.0.0.
ℹ Please use an explicit call to `as.character()` within `map_chr()` instead.
ℹ The deprecated feature was likely used in the OmnipathR package.
  Please report the issue at <https://github.com/saezlab/OmnipathR/issues>.”
One-to-many homolog matches: C4A; C4B; CCL27; CD200R1L; CD55; CEACAM1; CLEC10A; CSF2RB; DSG1; ENO1; GSTP1; HLA-DMB; HLA-E; HLA-F; IFNL3; IL11RA; IL22; KIR3DL3; KLRB1; KNG1; LILRB4; LYZ; MRGPRX2; PILRB; PRND; PRSS3; RNASE2; SAA1; SERPINA1



In [10]:
ortholog_resource = ortholog_resource %>% 
    rbind(c('S100a8', 'Mcam', '', '', 'cell_surface_ligand', 'manual', 'receptor', 'manual', '', 
            'https://doi.org/10.1007/s10585-016-9801-2')) %>%
    rbind(c('S100a9', 'Mcam', '', '', 'cell_surface_ligand', 'manual', 'receptor', 'manual', '', 
            'https://doi.org/10.1007/s10585-016-9801-2')) %>%
    rbind(c('S100a8', 'Alcam', '', '', 'cell_surface_ligand', 'manual', 'receptor', 'manual', '', 
            'https://doi.org/10.1007/s10585-016-9801-2')) %>%
    rbind(c('S100a9', 'Alcam', '', '', 'cell_surface_ligand', 'manual', 'receptor', 'manual', '', 
            'https://doi.org/10.1007/s10585-016-9801-2'))


In [11]:
if(!dir.exists("scdb")) dir.create("scdb/")
scdb_init("scdb/", force_reinit=T)
#> initializing scdb to testdb/

initializing scdb to scdb/



In [12]:
mat = scdb_mat("all_cells_liana")
mc_neutrophil = scdb_mc("neutrophil_mc")
mc_kinetics = scdb_mc("kinetics_mc")
pic_mc = scdb_mc("merged_neutrophil_epithelial_mc")

In [13]:
# RAN ALREADY

#fixed_pic_annots = c(mc_neutrophil@annots,
#                     ifelse(pic_mc@annots[9:length(pic_mc@annots)] %in% c('Alveolar spp l', 
#                                                                          'Alveolar spp1 h', 
#                                                                          'Alveolar secretory'),
#                            'Tumor cells',
#                            pic_mc@annots[9:length(pic_mc@annots)]))

#fixed_pic_colors = c(mc_neutrophil@colors[match(fixed_pic_annots, mc_neutrophil@annots)] %>% na.omit,
#                     mc_kinetics@colors[match(fixed_pic_annots, mc_kinetics@annots)] %>% na.omit)

#mcell_mc_add_annot("merged_neutrophil_epithelial_mc", fixed_pic_annots)
#mcell_mc_add_color("merged_neutrophil_epithelial_mc", fixed_pic_colors)

In [14]:
doublet_order = c(2, 16)
doublet_clusters = unique(mc_kinetics@annots)[doublet_order]
lymphoid_order = c(25, 26, 27, 21, 20, 28, 24)
myeloid_order = c(31, 32, 29, 34, 36, 33, 35, 22, 19, 23, 30)
epithelial_order = c(11, 13, 12, 14, 18, 15, 17, 10)
stromal_order = c(1, 3, 8, 9, 7)
fibroblast_order = c(5, 4, 6)
order_index = c(lymphoid_order, myeloid_order, epithelial_order, stromal_order, fibroblast_order)
cell_order = unique(mc_kinetics@annots)[order_index]

In [15]:
cell_order = cell_order[cell_order != "Neutrophils"]

In [16]:
cell_order

[1] "T cells"          "CD8 T cells"      "Treg"             "gd NKT"          
 [5] "NK cells"         "ILC2"             "B cells"          "Monocytes"       
 [9] "MonMacs"          "Perivasc macs"    "Ductal macs"      "cDC1"            
[13] "cDC2"             "pDC"              "Mreg DCs"         "Mast cells"      
[17] "Basophils"        "Progenitor 1"     "Progenitor 2"     "Alveolar"        
[21] "HS"               "Myoepithelial"    "Basal"            "MyoLum"          
[25] "Tumor cells"      "Endothelial"      "Mcam Endothelial" "Smooth muscle"   
[29] "Pericytes"        "Schwann cells"    "fibroblasts"      "Dpp4 fibroblasts"
[33] "CAFs"

In [17]:
sce = scm_export_mat_to_sce("all_cells_liana", add_log_counts=T)

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 12.4 GiB”


In [2075]:
good_pics = readLines('annotations/neutrophil_good_pics.txt')

In [2076]:
sum(good_pics %in% colnames(sce))

[1] 1190

In [20]:
length(good_pics)

[1] 1190

In [21]:
sce = sce[,colnames(sce) %in% union(union(names(mc_neutrophil@mc), 
                                          names(mc_kinetics@mc[mc_kinetics@annots[mc_kinetics@mc] %in% cell_order])),
                                    good_pics)]

In [22]:
sum(good_pics %in% colnames(sce))

[1] 1190

In [23]:
carcinoma_sce = sce[,sce$condition == 'tumor' & sce$Age %in% c('10w', '12w')] 

In [451]:
logcounts(carcinoma_sce) = as(logcounts(carcinoma_sce, withDimnames = FALSE), "dgCMatrix")

In [452]:
sum(grepl("__", carcinoma_sce$annots))

[1] 0

In [453]:
mle_res = read.delim("annotations/neutrophil_mle_res.txt", stringsAsFactors = F, row.names=1)

In [454]:
cell_order

[1] "T cells"          "CD8 T cells"      "Treg"             "gd NKT"          
 [5] "NK cells"         "ILC2"             "B cells"          "Monocytes"       
 [9] "MonMacs"          "Perivasc macs"    "Ductal macs"      "cDC1"            
[13] "cDC2"             "pDC"              "Mreg DCs"         "Mast cells"      
[17] "Basophils"        "Progenitor 1"     "Progenitor 2"     "Alveolar"        
[21] "HS"               "Myoepithelial"    "Basal"            "MyoLum"          
[25] "Tumor cells"      "Endothelial"      "Mcam Endothelial" "Smooth muscle"   
[29] "Pericytes"        "Schwann cells"    "fibroblasts"      "Dpp4 fibroblasts"
[33] "CAFs"

In [455]:
cancer_cells = c("Tumor cells")

In [456]:
tans = c('MHCII neut', 'Neut Ptgs2+', "Young TAN", "TAN1", "TAN2")

In [457]:
alpha = mle_res[good_pics, "alpha"]; names(alpha) = good_pics
epithelial_mc = mle_res[good_pics, "a_mc"]; names(epithelial_mc) = good_pics
immune_mc = mle_res[good_pics, "b_mc"]; names(immune_mc) = good_pics
merged_names = paste(ifelse(pic_mc@annots[immune_mc] %in% tans, "TANs", pic_mc@annots[immune_mc]), 
                     pic_mc@annots[epithelial_mc], 
                     sep = '__')
names(merged_names) = good_pics

In [458]:
tan_singlets = names(mc_neutrophil@mc[mc_neutrophil@annots[mc_neutrophil@mc] %in% tans])

In [459]:
cell_names = factor(ifelse(colnames(carcinoma_sce) %in% good_pics,
                           merged_names[colnames(carcinoma_sce)],
                           ifelse(colnames(carcinoma_sce) %in% names(mc_neutrophil@mc),
                                  ifelse(colnames(carcinoma_sce) %in% tan_singlets,
                                                  'TANs',
                                                  mc_neutrophil@annots[mc_neutrophil@mc[colnames(carcinoma_sce)]]),
                                  mc_kinetics@annots[mc_kinetics@mc[colnames(carcinoma_sce)]])))

In [460]:
cell_names %>% unique %>% sort

[1] Alveolar                    B cells                    
 [3] Basal                       Basophils                  
 [5] CAFs                        CD8 T cells                
 [7] cDC1                        cDC2                       
 [9] Cystatin neut               Cystatin neut__Progenitor 1
[11] Cystatin neut__Tumor cells  Dpp4 fibroblasts           
[13] Ductal macs                 Endothelial                
[15] fibroblasts                 gd NKT                     
[17] HS                          ILC2                       
[19] Mast cells                  Mcam Endothelial           
[21] MonMacs                     Monocytes                  
[23] Mreg DCs                    Myoepithelial              
[25] MyoLum                      NK cells                   
[27] pDC                         Pericytes                  
[29] Perivasc macs               Progenitor 1               
[31] Progenitor 2                Schwann cells              
[33] Smooth muscle               T cells                    
[35] TANs                        TANs__Alveolar             
[37] TANs__Basal                 TANs__HS                   
[39] TANs__Myoepithelial         TANs__MyoLum               
[41] TANs__Progenitor 1          TANs__Progenitor 2         
[43] TANs__Tumor cells           Treg                       
[45] Tumor cells                 Young neut                 
[47] Young neut__Progenitor 2    Young neut__Tumor cells    
48 Levels: Alveolar B cells Basal Basophils CAFs CD8 T cells cDC1 ... Young neut__Tumor cells

In [461]:
write_csv(data.frame(cell=colnames(carcinoma_sce), name=cell_names), "annotations/liana_cell_tanmerged_names.csv")

In [462]:
carcinoma_sce@colData$annots = cell_names

In [463]:
sum(good_pics %in% colnames(carcinoma_sce))

[1] 1118

In [464]:
sum(grepl("__", carcinoma_sce$annots))

[1] 1118

In [465]:
table(carcinoma_sce@colData$annots) %>% sort


                  Basophils Cystatin neut__Progenitor 1 
                          1                           1 
              Schwann cells                 TANs__Basal 
                          1                           1 
   Young neut__Progenitor 2                       Basal 
                          1                           2 
              Smooth muscle               Cystatin neut 
                          3                           4 
    Young neut__Tumor cells                  Young neut 
                          4                           5 
              Myoepithelial                    TANs__HS 
                          8                           8 
        TANs__Myoepithelial  Cystatin neut__Tumor cells 
                          9                          12 
                 Mast cells                          HS 
                         12                          18 
                   Mreg DCs                TANs__MyoLum 
                         18   

In [466]:
carcinoma_sce = carcinoma_sce[,table(carcinoma_sce@colData$annots)[carcinoma_sce@colData$annots] >= 10] # CHANGE was 30

Warning message in NSBS(i, x, exact = exact, strict.upper.bound = !allow.append, :
“subscript is an array, passing it thru as.vector() first”


In [467]:
carcinoma_sce$annots = droplevels(carcinoma_sce$annots)

In [438]:
liana_res <- liana_wrap(carcinoma_sce, idents_col = "annots", expr_prop=0.05,
                        resource = 'custom', # resource has to be set to 'custom' to work with external resources
                        external_resource = ortholog_resource , # provide orthologous resource
#                        method=c('sca', 'natmi') # run only with sca and natmi for comp. time
                        )


Running LIANA with `annots` as labels!

Warning message in exec(output, ...):
“10410 genes and/or 0 cells were removed as they had no counts!”
LIANA: LR summary stats calculated!

Now Running: Natmi

Now Running: Connectome

Now Running: Logfc

Now Running: Sca

Now Running: Cellphonedb



In [439]:
liana_res = liana_aggregate(liana_res)

Now aggregating natmi

Now aggregating connectome

Now aggregating logfc

Now aggregating sca

Now aggregating cellphonedb

Aggregating Ranks



In [440]:
write_csv(liana_res, "annotations/liana_results_tanmerged_tumor_pics_all_10min.csv")

In [474]:
saveRDS(carcinoma_sce, "h5ads/liana_carcinoma_tanmerged.RDS")